In [1]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks

In [2]:
from Analyze.results import Results_Analyze, plots_collection
from Preliminaries.concat_csv_files import concat_csv
from Spectrums.integral import simpson_integral_df

## Reading all files based on trigger

In [3]:
path = '../documents/single_muon/'

In [4]:
from Preliminaries.read_output_file import trigger_df
files = trigger_df(folders=path, rootFolder=True)

In [5]:
# df_10 = df.query('trigger == -10').reset_index(drop=True)
# df_20 = df.query('trigger == -20').reset_index(drop=True)
# df_40 = df.query('trigger == -40').reset_index(drop=True)

In [6]:
# files_10 = files.query('trigger == -10')
# files_20 = files.query('trigger == -20')
# files_40 = files.query('trigger == -40')

In [7]:
files

,path,trigger
0,../documents/single_muon/1619201634.9231706,-40.0
1,../documents/single_muon/1619246158.7057853,-40.0
2,../documents/single_muon/1619297657.0683234,-10.0
3,../documents/single_muon/1619314826.6753685,-20.0
4,../documents/single_muon/1619481562.3015544,-20.0
5,../documents/single_muon/1619540293.2577727,-10.0
6,../documents/single_muon/1619641030.8896358,-40.0


In [8]:
dfs = []
for i in range(files.shape[0]):
    df = concat_csv(path=files['path'][i])
    dfs.append(df)

C:\.Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3357: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [9]:
waveforms = []
for i in range(files.shape[0]):
    waveform = dfs[i][1:]
    waveforms.append(waveform)

In [10]:
from Preliminaries.read_output_file import retrieve_y_to_volts, convert_y_to_volts

In [11]:
converter = retrieve_y_to_volts(path=files['path'][0])
converter

,y_zero,y_mult,y_off
values,0.0,0.0004,50.0


In [12]:
for i in range(len(waveforms)):
    waveforms[i] = convert_y_to_volts(waveform=waveforms[i], converter_df=converter)

In [13]:
waveforms[4]

,event_0,event_1,event_2,event_3,event_4,event_5,event_6,event_7,event_8,event_9,...,event_20688,event_20689,event_20690,event_20691,event_20692,event_20693,event_20694,event_20695,event_20696,event_20697
0,0.8,0.8,0.8,1.2,1.2,1.6,-1.6,0.8,0.8,0.4,...,0.0,-0.8,1.2,0.0,0.0,0.8,0.8,0.8,2.0,1.6
1,1.6,2.0,0.8,1.6,1.6,1.2,0.0,0.8,0.4,0.8,...,0.4,2.4,1.2,1.2,0.0,1.2,0.8,1.6,-0.4,1.2
2,1.6,0.8,1.2,0.8,1.2,1.6,2.4,1.2,1.2,1.6,...,0.4,1.2,0.8,1.6,1.6,0.0,0.8,1.2,1.6,1.2
3,1.2,0.8,0.8,0.8,1.2,1.6,1.2,0.4,0.8,1.2,...,0.4,1.2,0.4,1.6,1.2,1.2,0.8,0.8,1.2,0.8
4,0.4,1.2,0.4,1.2,1.6,0.8,-0.4,1.6,0.8,1.6,...,0.8,1.2,0.8,1.2,0.4,1.2,0.8,0.8,0.4,0.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,1.2,0.4,1.2,1.2,0.4,0.8,1.2,1.6,0.4,0.4,...,0.8,1.6,0.8,1.2,0.8,1.2,1.2,0.8,0.8,0.8
2496,0.8,0.0,1.6,1.2,1.6,1.6,1.2,0.0,1.2,1.6,...,0.4,-0.8,0.4,0.4,0.8,1.6,0.4,0.8,1.2,0.8
2497,1.2,0.4,0.4,0.4,0.8,0.4,0.8,1.2,1.6,1.2,...,0.8,1.2,0.8,1.6,1.6,0.4,1.2,1.2,1.6,1.6
2498,1.2,1.2,1.2,1.2,1.6,1.6,0.8,0.8,1.2,0.8,...,0.4,2.4,0.8,0.8,1.2,0.4,0.8,0.4,1.2,1.6


## Concatenating all files based on trigger

In [14]:
# %%time
# df_10 = pd.concat( 
#     [concat_csv(path=files_10['path'].iloc[i]) for i in range(files_10.shape[0])]
#     , axis=1 )
# df_10.columns = ['event_' + str(i) for i in range(df_10.shape[1])]

# df_20 = pd.concat( 
#     [concat_csv(path=files_20['path'].iloc[i]) for i in range(files_20.shape[0])]
#     , axis=1 )
# df_20.columns = ['event_' + str(i) for i in range(df_20.shape[1])]

# df_40 = pd.concat( 
#     [concat_csv(path=files_40['path'].iloc[i]) for i in range(files_40.shape[0])]
#     , axis=1 )
# df_40.columns = ['event_' + str(i) for i in range(df_40.shape[1])]

## Converting values to mV

the conversion values are the same, so no worries

In [16]:
# waveforms_10 = df_10[1:]
# waveforms_20 = df_20[1:]
# waveforms_40 = df_40[1:]

In [17]:
# converter = y_to_volts(path=files_10['path'].iloc[0])
# converter

In [18]:
# waveforms_10 = converter['y_zero'][0] + converter['y_mult'][0]*(waveforms_10 - converter['y_off'][0])
# waveforms_10 *= 1_000 # mV

# waveforms_20 = converter['y_zero'][0] + converter['y_mult'][0]*(waveforms_20 - converter['y_off'][0])
# waveforms_20 *= 1_000 # mV

# waveforms_40 = converter['y_zero'][0] + converter['y_mult'][0]*(waveforms_40 - converter['y_off'][0])
# waveforms_40 *= 1_000 # mV

In [19]:
# baseLine = waveforms_40.iloc[150:].mean().mean()
baseLine = 0.9687394555270057 #mV
# height = 1000*( converter['y_zero'][0] + converter['y_mult'][0]*(0 - converter['y_off'][0]) )
height = -20 #mV
#
saturacao = -70.8 #mV

In [20]:
# baseLine = 52.41176078011208
# saturacao = -179.41176078011208

# df_10 -= baseLine
# df_20 -= baseLine
# df_40 -= baseLine

In [21]:
# lista = [i for i in range(2500)]
# plt.plot(lista, waveforms_40['event_20000'])

In [22]:
# peaks, _ = find_peaks(-1*waveforms_40['event_6587'], height=20)
# peaks

## Retrieving data results

Note:

In order to calculate the integrals, the base line must be the most possible close to zero. It may looks like we are changing the value of the integral, but, in reality, we are only changing the values in order to calculate the variation, which, in reality, is given by subtracting a constant.

In essence: just subtract the baseLine constant in order to put the values around zero.

In [23]:
def peaks_single_muon(df, height, first_peak_loc=False):

    if first_peak_loc != False:
        if type(first_peak_loc) != int:
            raise ValueError('first_peak_loc must be False or int, only')


    Peaks = pd.DataFrame()
    problems = pd.DataFrame()

    for i in range(df.shape[1]):

        event = df[df.columns[i]]
        x_peaks, _ = find_peaks(-1*event, height=height)

        if len(x_peaks) != 1:
            # raise ValueError(f'the numbers of peaks must be 1 and it found {len(x_peaks)} on {event.name}')
            problems[event.name] = ['peaks quantity']
            continue

        if first_peak_loc != False: #trigger makes us expect the pulse in [80:120]
            if (x_peaks[0] < first_peak_loc - 15) or (x_peaks[0] > first_peak_loc + 15):
                # raise ValueError(f'the first peak of {event.name} is not around the point {first_peak_loc}, but in the instant {x_peaks[0]}')
                problems[event.name] = ['1st peak outrange']
                continue

        y_peaks = event.iloc[x_peaks].tolist()

        peaks = np.append(x_peaks, y_peaks)

        Peaks[df.columns[i]] = peaks

    Peaks = Peaks.T
    Peaks.columns = ['peak_X0', 'peak_Y0']

    problems = problems.T
    if problems.shape[1] > 0:
        problems.columns = ['problem']

    return(Peaks, problems)



def contours_single_muon(waveform, peak, random_left=10, random_right=15):

    pulses_0 = pd.DataFrame()
    
    for i in range(waveform.shape[1]):
        try:
            event = waveform[waveform.columns[i]]

            peak_X0 = int(peak['peak_X0'][i])

            limits_0 = [ peak_X0-random_left, peak_X0+random_right ]

            pulse_0 = event.iloc[ limits_0[0]:limits_0[1] ].to_list()

            pulses_0[ event.name ] = pulse_0
        except:
            raise ValueError(f'problem on {event.name}')

    return(pulses_0)



def single_muon_analysis(waveforms, height, baseLine=0):

    peaks     = peaks_single_muon(df=waveforms, height=height)

    integrals = simpson_integral_df(contours_single_muon(waveform=waveforms-baseLine, peak=peaks))

    return(peaks, integrals)

In [24]:
def convert_charge(integral):
    
    R = 50 #ohm
    binTime = 4E-3 #micro-sec
    VoltCh  = 100 / 255 #100mV/(256-1)bits

    charge = 1_000*integral*binTime*VoltCh/R #pC

    return(charge)

## Hands on

trigger = -10 mV

In [ ]:
# %%time
# peaks_10, problems_10 = peaks_single_muon(df=waveforms_10, height=-1*height, first_peak_loc=100)

In [ ]:
# plt.plot(lista, waveforms_10[problems_10.index])

In [ ]:
# %%time
# contours_10 = contours_single_muon(waveform=waveforms_10[peaks_10.index], peak=peaks_10, random_left=10, random_right=15)

In [ ]:
# integral_10 = simpson_integral_df(contours_10-baseLine)

trigger = -20 mV

In [ ]:
# %%time
# peaks_20, problems_20 = peaks_single_muon(df=waveforms_20, height=-1*height, first_peak_loc=100)

In [ ]:
# problems_20

In [ ]:
#plt.plot(lista, waveforms_20[problems_20.index])

In [ ]:
# %%time
# contours_20 = contours_single_muon(waveform=waveforms_20[peaks_20.index], peak=peaks_20, random_left=10, random_right=15)

In [ ]:
# integral_20 = simpson_integral_df(contours_20-baseLine)

trigger = -40 mV

In [ ]:
# %%time
# peaks_40, problems_40 = peaks_single_muon(df=waveforms_40, height=-1*height, first_peak_loc=100)

In [ ]:
# problems_40

In [ ]:
# %%time
# contours_40 = contours_single_muon(waveform=waveforms_40[peaks_40.index], peak=peaks_40, random_left=10, random_right=15)

In [ ]:
# integral_40 = simpson_integral_df(contours_40-baseLine)

In [ ]:
# peaks_0, problems_0 = peaks_single_muon(df=waveforms[0], height=-1*height, first_peak_loc=100)

In [25]:
%%time
peaks = []
problems = []
for i in range(files.shape[0]):
    pk, pb = peaks_single_muon(df=waveforms[i], height=-1*height, first_peak_loc=100)
    peaks.append(pk)
    problems.append(pb)

Wall time: 2min 42s


In [30]:
# for i in range(files.shape[0]):
#     print(peaks[i].shape[0])

In [39]:
for i in range(files.shape[0]):
    path = files['path'][i]
    print(path)
    os.mkdir(f"{path}/results")
    peaks[i].to_csv( f"{path}/results/peaks.csv" )
    problems[i].to_csv( f"{path}/results/problems.csv" )

../documents/single_muon/1619201634.9231706
../documents/single_muon/1619246158.7057853
../documents/single_muon/1619297657.0683234
../documents/single_muon/1619314826.6753685
../documents/single_muon/1619481562.3015544
../documents/single_muon/1619540293.2577727
../documents/single_muon/1619641030.8896358


In [41]:
%%time
contours_0 = contours_single_muon(waveform=waveforms[0][peaks[0].index], peak=peaks[0], random_left=10, random_right=15)

Wall time: 5.37 s


In [43]:
simpson_integral_df(contours_0 - baseLine)

,0
event_0,-196.449747
event_1,-357.516414
event_2,-184.049747
event_3,-266.716414
event_4,-312.449747
...,...
event_5896,-305.783080
event_5897,-175.516414
event_5898,-196.449747
event_5899,-193.249747


In [49]:
%%time
contours = []
integrals = []
for i in range(files.shape[0]):

    ct = contours_single_muon(waveform=waveforms[i][peaks[i].index], peak=peaks[i], random_left=10, random_right=15)
    contours.append(ct)

    it = simpson_integral_df(ct-baseLine)
    integrals.append(it)

Wall time: 2min 22s


In [51]:
for i in range(files.shape[0]):
    path = files['path'][i]
    print(path)
    #os.mkdir(f"{path}/results")
    contours[i].to_csv( f"{path}/results/contours.csv" )
    integrals[i].to_csv( f"{path}/results/integrals.csv" )

../documents/single_muon/1619201634.9231706
../documents/single_muon/1619246158.7057853
../documents/single_muon/1619297657.0683234
../documents/single_muon/1619314826.6753685
../documents/single_muon/1619481562.3015544
../documents/single_muon/1619540293.2577727
../documents/single_muon/1619641030.8896358


In [ ]:
# peaks_10, integral_10 = single_muon_analysis(waveforms=waveforms_10, height=height, baseLine=baseLine)
# peaks_20, integral_20 = single_muon_analysis(waveforms=waveforms_20, height=height, baseLine=baseLine)
# peaks_40, integral_40 = single_muon_analysis(waveforms=waveforms_40, height=height, baseLine=baseLine)

In [ ]:
# integral_10 = convert_charge(integral=integral_10)
# integral_20 = convert_charge(integral=integral_20)
# integral_40 = convert_charge(integral=integral_40)

In [ ]:
# Main plot
fig, axes = plt.subplots(ncols=3, nrows=2, figsize=(20,14))
fig.suptitle('title', fontsize=18)

# Fig 1
sns.histplot( peaks_10['peak_Y0'], ax=axes[0,0], color='orange' )
axes[0,0].set_title(f'Espectro de picos; Trigger = -10mV; {peaks_10.shape[0]} events')
axes[0,0].set_xlabel('peak (mV)')

# Fig 2
sns.histplot( peaks_20['peak_Y0'], ax=axes[0,1], color='yellow' )
axes[0,1].set_title(f'Espectro de picos; Trigger = -20mV; {peaks_20.shape[0]} events')
axes[0,1].set_xlabel('peak (mV)')

# Fig 3
sns.histplot( peaks_40['peak_Y0'], ax=axes[0,2], color='gray' )
axes[0,2].set_title(f'Espectro de picos; Trigger = -40mV; {peaks_40.shape[0]} events')
axes[0,2].set_xlabel('peak (mV)')

# Fig 4
sns.histplot( integral_10[integral_10.columns[0]], ax=axes[1,0], color='orange' )
axes[1,0].set_title(f'Espectro de carga; Trigger = -10mV; {integral_10.shape[0]} events')
axes[1,0].set_xlabel('charge (pC)')

# Fig 5
sns.histplot( integral_20[integral_20.columns[0]], ax=axes[1,1], color='yellow' )
axes[1,1].set_title(f'Espectro de carga; Trigger = -20mV; {integral_20.shape[0]} events')
axes[1,1].set_xlabel('charge (pC)')

# Fig 6
sns.histplot( integral_40[integral_40.columns[0]], ax=axes[1,2], color='gray' )
axes[1,2].set_title(f'Espectro de carga; Trigger = -40mV; {integral_40.shape[0]} events')
axes[1,2].set_xlabel('charge (pC)')

plt.show()

In [ ]:
x = np.arange(2500)
event = 'event_4395'
#plt.plot(x, df_10[event])

In [ ]:
peaks_10_filt = peaks_10.query('peak_Y0 > @saturacao')
peaks_20_filt = peaks_20.query('peak_Y0 > @saturacao')
peaks_40_filt = peaks_40.query('peak_Y0 > @saturacao')

integral_10_filt = integral_10.loc[ peaks_10_filt.index.to_list() ]
integral_20_filt = integral_20.loc[ peaks_20_filt.index.to_list() ]
integral_40_filt = integral_40.loc[ peaks_40_filt.index.to_list() ]

In [ ]:
# Main plot
fig, axes = plt.subplots(ncols=3, nrows=2, figsize=(20,14))
fig.suptitle('Filtragem: sem saturação', fontsize=18)

# Fig 1
sns.histplot( -1*peaks_10_filt['peak_Y0'], ax=axes[0,0], color='orange', bins=50 )
axes[0,0].set_title(f'Espectro de picos; Trigger = -10mV; {peaks_10_filt.shape[0]} events')
axes[0,0].set_xlabel('peak (mV)')

# Fig 2
sns.histplot( -1*peaks_20_filt['peak_Y0'], ax=axes[0,1], color='yellow', bins=50 )
axes[0,1].set_title(f'Espectro de picos; Trigger = -20mV; {peaks_20_filt.shape[0]} events')
axes[0,1].set_xlabel('peak (mV)')

# Fig 3
sns.histplot( -1*peaks_40_filt['peak_Y0'], ax=axes[0,2], color='gray', bins=50 )
axes[0,2].set_title(f'Espectro de picos; Trigger = -40mV; {peaks_40_filt.shape[0]} events')
axes[0,2].set_xlabel('peak (mV)')

# Fig 4
sns.histplot( -1*integral_10_filt[integral_10.columns[0]], ax=axes[1,0], color='orange', bins=100 )
axes[1,0].set_title(f'Espectro de carga; Trigger = -10mV; {integral_10_filt.shape[0]} events')
axes[1,0].set_xlabel('charge (pC)')

# Fig 5
sns.histplot( -1*integral_20_filt[integral_20.columns[0]], ax=axes[1,1], color='yellow', bins=100 )
axes[1,1].set_title(f'Espectro de carga; Trigger = -20mV; {integral_20_filt.shape[0]} events')
axes[1,1].set_xlabel('charge (pC)')

# Fig 6
sns.histplot( -1*integral_40_filt[integral_40.columns[0]], ax=axes[1,2], color='gray', bins=120 )
axes[1,2].set_title(f'Espectro de carga; Trigger = -40mV; {integral_40_filt.shape[0]} events')
axes[1,2].set_xlabel('charge (pC)')

plt.show()

In [ ]:
# Main plot
# fig, axes = plt.subplots(ncols=2, nrows=1, figsize=(20,14))
# fig.suptitle('title', fontsize=18)

# Fig 6
sns.histplot( -1*integral_40[integral_40.columns[0]], bins=120, color='orange' )

# Fig 6
sns.histplot( -1*integral_40_filt[integral_40.columns[0]], bins=120, color='yellow' )

